# 19章 1つの名義変数で量的予測変数を予測する

## はじめに
- 本章で扱うのは、量的被予測変数と名義的予測変数のデータ構造
- 例
    - 政治的な党派関係　→ 収入を予測
    - 性行動のカテゴリ　→　寿命を予測（今回扱うこと）
- このような種類のデータ構造に関する伝統的な手法
    - 一要因分散分析（ANOVA）、または一元配置ANOVAと呼ばれている
    - これらの手法を用いて、予測変数によって、被予測変数の平均値に相違があるのかを調べる。
        - 政治的な党派ごとに収入が異なるか、など
- 今回紹介するベイジアンアプローチは、この伝統的な分散分析モデルを階層的に一般化したもの
- また、量的予測変数に伴う上位の名義的予測変数が存在する状況を考える。
    - このようなデータ構造については、伝統的な手法として共分散分析(ANCOVA)が用いられている。
    - こちらについても、伝統的なモデルの一般化を考える

- 1つの名義的予測変数の線形関数
- データのノイズを表すための、正規分布に従う高騰のリンク関数も含む

- 一要因分散分析、一元配置ANOVAについて説明できると良い。。
- 共分散分析も。。


## 19.1. 量的データの複数の群の表記

<img src="./img/fig19_1.png" width="500">

- 各群のデータは「中心傾向周りの」「ランダムな変動」として記述されるものを扱う
- 各群の中心傾向は「全体のベースライン($\beta_0$)からの偏向」として捉える
- なお、ここではデータは、標準偏差の等しい正規分布に従っているとする(後ほど、標準偏差が群ごとに異なる場合を考える)

- 各群ごとの予測値 $\mu$ は、下記の式で表される。全体のベースラインに群の変更を加えている
$$
\begin{aligned} \mu & = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } \\ & = \beta _ { 0 } + \vec { \beta } \cdot \vec { x } \end{aligned}
$$

- また、全体のベースラインからのカテゴリの偏向は、足しあげると0になるようになっている。（ゼロ和条件）

$$
\sum _ { j } \beta _ { [ j ] } = 0
$$

- ゼロ和条件がないと完全ではない。ゼロ和条件を満たすようにするのは簡単(式19.3)

- 図19.1　で示した記述的モデルは、古典的なANOVAで用いた伝統的なもの。
- より一般的なモデルはベイズようソフトウェアで直接実装される。
    - 外れ値は正規分布の代わりに裾の重いノイズ分布(t分布など)を用いることによって取り込み可能
    - 異なる群に異なる標準偏差を与えることができる

## 19.2. 伝統的な分散分析
- 「分散分析」：データ全体の分散を郡内分散と群間分散に分解するという意味
- 代数的にいうと、
    - 「全体平均値からの偏差の二乗和」＝「それぞれの群平均値からの偏差の二乗和」＋「全体平均値からの群平均の偏差の二乗和」
    - つまり、「全体の分散」＝「郡内分散」＋「群間分散」
    - ANOVAはこれらの代数的な側面を表している。
- ベイズ的手法は、要素の分散を推定できはするが、このような代数的な関係は使わない。
- そのため、ベイジアンアプローチとANOVAは異なるものである。しかし、類似はしている。

### 伝統的なANOVAについて
- 帰無仮説：各群の平均は等しい（なお、ANOVAは3群以上の場合に用いる）
- p値に基づいて上記を決定する
- p値は、帰無仮説からの擬似サンプリングによって計算される
- なお、伝統的なANOVAでは、帰無仮説は下記を前提としている
    1. 各群のデータが正規分布に従っている
    2. 各群、同じ標準偏差であること（分散の等質性）
- サンプリングする統計量は、郡内分散と群間分散の比。これをF比と呼ぶ。サンプリング分布はF分布と呼ばれている。
- この2つの前提は慣例として定着しているため、本書の基本モデルでも前提として持ってくる。
- ベイズ用ソフトウェアではこれらの前提は緩和される。
    - 各群によって異なる分散を用いることができる
    - 正規分布以外も郡内のデータを表すのに用いることができる

# 19.3. 階層ベイジアン・アプローチ
- 再掲
<img src="./img/fig19_1.png">

- 私たちの目標：ベイズ的な枠組みで、上述したパラメータを推定することである
- 式19.1.
$$
\begin{aligned} \mu & = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } \\ & = \beta _ { 0 } + \vec { \beta } \cdot \vec { x } \end{aligned}
$$

- そのため、全てのパラメータは図19.2のように、構造化された事前分布を与えられる必要がある。

### 階層的ダイアグラム
<img src="./img/fig19_2.png" width="500">

- 下から見ていく
- データ $y_i$が予測値 $\mu_i$を中心に、正規分布に従っている。その正規分布の標準偏差には$\sigma_y$には一様事前分布が与えられている
- ベースラインパラメータ $\beta_0$はデータにとって広い範囲の正規事前分布を与える
- 群の偏向パラメータ $\beta_j$には平均0の正規事前分布を与える
    - なぜなら偏向パラメータの総和は0になるため。

- ベイジアンアプローチでの目新しいところは $\sigma_{\beta}$ 。それを偏向パラメータ分布の標準偏差とすること。
- 事前分布を何か選ぶこともできるし、定数として設定することも可能。
    - $\sigma_{\beta}$を大きな定数として設定することで、伝統的なANOVAにもっとも近い結果となる
- ???

### 19.3.1. JAGSでの実装


### 19.3.2. 例：セックスと死亡について
- ショウジョウバエでの実験
- オスについて性行為がどう負担になるかを調べる。
- 5つの群について、25匹ずつのオスのショウジョウバエが割り当てられた
    - ゼロ：相手がいないオスの群
    - 妊娠1：妊娠したメス1匹のみとつがいにされたオスの群
    - 妊娠8：妊娠したメス8匹とつがいにされた群
    - 処女1：未交尾のめす